# START

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly.express as px
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('uk_clustered.csv')
df['Cluster'] = df['Cluster'].astype('category')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


In [3]:
df

,Customer ID,Invoice,Description,Quantity,InvoiceDate,Cancelled,Total Amount,Recency,Frequency,Monetary,Cluster
0,13085,489434,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,False,83.40,335.0,6.0,2017.20,Occasionals
1,13085,489434,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,False,81.00,335.0,6.0,2017.20,Occasionals
2,13085,489434,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,False,81.00,335.0,6.0,2017.20,Occasionals
3,13085,489434,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,False,100.80,335.0,6.0,2017.20,Occasionals
4,13085,489434,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,False,30.00,335.0,6.0,2017.20,Occasionals
...,...,...,...,...,...,...,...,...,...,...,...
371522,17530,538171,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,False,5.90,21.0,4.0,724.26,Regulars
371523,17530,538171,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,False,3.75,21.0,4.0,724.26,Regulars
371524,17530,538171,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,False,3.75,21.0,4.0,724.26,Regulars
371525,17530,538171,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,False,7.50,21.0,4.0,724.26,Regulars


# Cluster Analysis: trends by date, favorite items

In [4]:
df['Month'] = df['InvoiceDate'].dt.month

In [5]:
monthly = df.groupby(['Cluster', 'Month'])['Total Amount'].mean().reset_index()
fig = px.bar(monthly, x='Month', y='Total Amount', color='Cluster', template='seaborn', barmode='group', title='Mean Amount per Month')
fig.show()

In [6]:
monthly_sum = df.groupby(['Cluster', 'Month'])['Total Amount'].sum().reset_index()
fig = px.bar(monthly_sum, x='Month', y='Total Amount', color='Cluster', template='seaborn', barmode='group', title='Total Amount per Month')
fig.show()

Here we see the first curious finding about our british customers. Although the mean amount per month is higher for Loyals, who are considered high-monetary and high-frequency, the aggregated amount of monthly purchases for regulars is much higher, especially in Autumn months (see the sum barchart above). This means that regular customers make the most of the revenue for the business.

In [7]:
customer_count = df.groupby('Cluster')['Customer ID'].nunique().reset_index()
customer_count.rename(columns={'Customer ID': 'Customer Count'}, inplace=True)

fig = px.bar(
    customer_count,
    x='Cluster',
    y='Customer Count',
    color='Cluster',
    template='seaborn',
    title='Number of Customers per Cluster'
)
fig.show()

We can see how the number of the Regulars is almost 6-fold compared to the loyals.

In [8]:
timeframe = df.groupby(['Cluster', 'InvoiceDate'])['Total Amount'].sum().reset_index().sort_values(by="InvoiceDate")
fig = px.scatter(timeframe, x='InvoiceDate', y='Total Amount', color='Cluster', template='xgridoff')
fig.show()

# Items analysis

In [9]:
df['Description'] = df['Description'].str.lower()

In [10]:
df['Description'].value_counts()

Description
white hanging heart t-light holder    3092
regency cakestand 3 tier              1662
strawberry ceramic trinket box        1387
assorted colour bird ornament         1289
home building block word              1164
                                      ... 
number tile cottage garden, 3            1
number tile cottage garden, 4            1
number tile cottage garden, 6            1
number tile cottage garden, 7            1
baking mould easter egg milk choc        1
Name: count, Length: 4426, dtype: int64

In [11]:
pop_items = df.groupby('Description')['Quantity'].sum().sort_values(ascending = False)
pop_items = pd.DataFrame(pop_items)
pop_items
fig = px.bar(pop_items.head(15), x=pop_items.head(15).index, y='Quantity', template='ggplot2', color='Quantity')
fig.show()

In [12]:
pop_items_per_cluster = df.groupby(['Cluster','Description'])['Invoice'].nunique()
pop_items_per_cluster = pop_items_per_cluster.reset_index()
pop_items_per_cluster = pop_items_per_cluster.sort_values(['Cluster', 'Invoice'], ascending=[True, False])

top_10_per_cluster = pop_items_per_cluster.reset_index().groupby('Cluster').head(10)
top_10_per_cluster

,index,Cluster,Description,Invoice
0,4243,Loyals,white hanging heart t-light holder,382
1,4323,Loyals,wooden frame antique white,203
2,3233,Loyals,regency cakestand 3 tier,190
3,2232,Loyals,lunch bag red spotty,173
4,4328,Loyals,wooden picture frame white finish,172
5,4310,Loyals,wood s/3 cabinet ant white finish,171
6,2056,Loyals,jumbo bag red retrospot,158
7,1285,Loyals,edwardian parasol black,151
8,2049,Loyals,jumbo bag baroque black white,148
9,3226,Loyals,red woolly hottie white heart.,148


In [13]:
#favorite items per cluster
clusters = top_10_per_cluster['Cluster'].unique()
for cluster in clusters:
    fig = px.bar(top_10_per_cluster[top_10_per_cluster['Cluster'] == cluster], 
                 x = 'Description', 
                 y='Invoice', 
                 title = f'Top 10 items for {cluster}', 
                 template = 'ggplot2',
                 color = 'Invoice')
    fig.show()

In [14]:
exp_items_per_cluster = df.groupby(['Cluster','Description'])['Quantity'].sum()
exp_items_per_cluster = exp_items_per_cluster.reset_index()
exp_items_per_cluster = exp_items_per_cluster.sort_values(['Cluster', 'Quantity'], ascending=[True, False])

top_10_exp_per_cluster = exp_items_per_cluster.reset_index().groupby('Cluster').head(10)
top_10_exp_per_cluster

,index,Cluster,Description,Quantity
0,687,Loyals,brocade ring purse,33653
1,4348,Loyals,world war 2 gliders asstd designs,21312
2,262,Loyals,assorted colour bird ornament,18702
3,3833,Loyals,small popcorn holder,14103
4,4243,Loyals,white hanging heart t-light holder,12782
5,2056,Loyals,jumbo bag red retrospot,11917
6,3106,Loyals,red harmonica in box,9839
7,2614,Loyals,pack of 72 retro spot cake cases,9482
8,2060,Loyals,jumbo bag strawberry,8227
9,3350,Loyals,rotating silver angels t-light hldr,7950
